In [57]:

from PIL import Image
import PIL.Image as pil
import numpy as np
from numpy.matlib import repmat
import matplotlib.pyplot as plt
import torch
from skimage import io
import os
import json
import copy
import glob
import cv2
from numpy.matlib import repmat
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm

import torch.nn.functional as F
import torch
from tqdm import tqdm
%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: {}".format(device))
import math
import os
os.chdir('../..')
from torch.utils.data import Dataset, DataLoader
# Data structures and functions for rendering
from pytorch3d.structures import Pointclouds
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLOrthographicCameras, 
    PointsRasterizationSettings,
    OpenGLPerspectiveCameras,
    SfMPerspectiveCameras,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor
)

FOLDER_mpc_rgb = "./synthesis_rgb"
FOLDER_mpc_disp = "./synthesis_disp"
FOLDER_out = "./synthesis_out"

Device: cuda


In [58]:
def disp_to_depth(disp, min_depth, max_depth):
    """Convert network's sigmoid output into depth prediction
    The formula for this conversion is given in the 'additional considerations'
    section of the paper.
    """
    min_disp = 1 / max_depth
    max_disp = 1 / min_depth
    scaled_disp = min_disp + (max_disp - min_disp) * disp
    depth = 1 / scaled_disp
    return scaled_disp, depth

def read_rgb(rgb_file):
    
    #rgb = io.imread(rgb_file)[:,:,:3]
    im = Image.open(rgb_file)
    
    # Resize for mpc
    width, height = im.size  
    new_width = 600
    new_height = 600
    
    im = im.resize((new_width, new_height))
    rgb = np.array(im)[:,:,:3]
    
    return rgb

def read_depth(depth_file):
    
    '''depth = io.imread(depth_file)
    # Reference: https://carla.readthedocs.io/en/stable/cameras_and_sensors/#camera-depth-map
    depth = depth[:, :, 0] * 1.0 + depth[:, :, 1] * 256.0 + depth[:, :, 2] * (256.0 * 256)
    depth = depth * (1/ (256 * 256 * 256 - 1))'''
    
    disp = np.load(depth_file)
    disp = torch.from_numpy(disp)
    disp = torch.nn.functional.interpolate(disp, (600,600),mode="nearest")
    disp = disp.reshape((disp.shape[2],disp.shape[3]))
    
    _, depth = disp_to_depth(disp, 0.1, 100)
    sc = 3.6
    #sc = 1.45
    depth = depth * sc
    depth = depth.cpu().detach().numpy()
    
    return depth

def get_rendered_image(depth, img, rel_pose12, K):
    
    rel_pose12 = rel_pose12.copy()
    rel_pose12[0:2,3] *= -1
    angles = rotationMatrixToEulerAngles(rel_pose12[:3,:3])
    rel_pose12[:3,:3] = eulerAnglesToRotationMatrix(angles * np.array([1.0,1.0, -1.0]))

    verts, color = depth_to_local_point_cloud(depth, color=img,k = K)
    
    # See also: 
    # https://github.com/facebookresearch/pytorch3d/blob/stable/docs/tutorials/render_colored_points.ipynb
    
    verts = torch.Tensor(verts).to(device)
    rgb = torch.Tensor(color).to(device)
    point_cloud = Pointclouds(points=[verts], features=[rgb])
    
    R = torch.from_numpy(rel_pose12[:3,:3].astype(np.float32)).unsqueeze(0)
    T = torch.FloatTensor(1*rel_pose12[:3,3].astype(np.float32)).unsqueeze(0)
    cameras = SfMPerspectiveCameras(device=device, R=R, T=T,
                                        focal_length = torch.FloatTensor([[1,1]]),
                                       principal_point = torch.FloatTensor([[0,0]]))

    raster_settings = PointsRasterizationSettings(
            image_size=img.shape[1] ,
            radius = 0.01,
            points_per_pixel = 100
        )
    
    renderer = PointsRenderer(
            rasterizer=PointsRasterizer(cameras=cameras, raster_settings=raster_settings),
            compositor=AlphaCompositor()
        )

    images = renderer(point_cloud)
    return images[0, ..., :3].cpu().numpy()

def depth_to_local_point_cloud(image, color=None, k = np.eye(3),max_depth=1.1):
    """
    Convert an image containing CARLA encoded depth-map to a 2D array containing
    the 3D position (relative to the camera) of each pixel and its corresponding
    RGB color of an array.
    "max_depth" is used to omit the p+oints that are far enough.
    Reference: 
    https://github.com/carla-simulator/driving-benchmarks/blob/master/version084/carla/image_converter.py
    """
    far = 1000.0  # max depth in meters.
    normalized_depth = image# depth_to_array(image)
    height, width = image.shape

    # 2d pixel coordinates
    pixel_length = width * height
    u_coord = repmat(np.r_[width-1:-1:-1],
                     height, 1).reshape(pixel_length)
    v_coord = repmat(np.c_[height-1:-1:-1],
                     1, width).reshape(pixel_length)

    if color is not None:
        color = color.reshape(pixel_length, 3)
    normalized_depth = np.reshape(normalized_depth, pixel_length)

    # Search for pixels where the depth is greater than max_depth to
    # delete them

    # pd2 = [u,v,1]
    p2d = np.array([u_coord, v_coord, np.ones_like(u_coord)])

    # P = [X,Y,Z]
    p3d = np.dot(np.linalg.inv(k), p2d)
    p3d *= normalized_depth 
    p3d = np.transpose(p3d, (1,0))
    return p3d, color / 255.0


Reference: https://learnopencv.com/rotation-matrix-to-euler-angles/

In [59]:
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rotationMatrixToEulerAngles(R) :

    assert(isRotationMatrix(R))
    
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    
    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])*180/np.pi

# Calculates Rotation Matrix given euler angles.
def eulerAnglesToRotationMatrix(theta) :
    
    theta *= np.pi / 180
    
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])
        
        
                    
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])
                
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])
                    
                    
    R = np.dot(R_x, np.dot( R_y, R_z ))

    return R

Example showing how image rendering works (shifting the camera's position horizontally)

In [ ]:
id_list_pos = os.listdir(FOLDER_mpc_rgb)
id_list_pos.sort()
# Add folders to be synthesized to the list 
id_list_pos = ["pos178"]                      
cam_rgb = "center"
cam_disp = "disp_center"
translation_arr = [-0.25, 0.25, -0.50, 0.50, -0.75, 0.75, -1.0, 1.0, -1.25, 1.25, -1.50, 1.50]
for x in range(0, len(translation_arr)):
    for pos in range(0, len(id_list_pos)):
        position = id_list_pos[pos]
        print("Translation is: {}".format(translation_arr[x]))
        id_list_frames = os.listdir(os.path.join(FOLDER_mpc_rgb, position, "gt", cam_rgb))
        id_list_frames.sort()
        for fr in range(0, len(id_list_frames)):
            frame_no = id_list_frames[fr]
            rgb_file1_mpc   = os.path.join(FOLDER_mpc_rgb, position, "gt", cam_rgb, frame_no)
            depth_file1_mpc = os.path.join(FOLDER_mpc_disp, position, cam_disp, frame_no[:-4] + ".npy")
            
            K = np.array([[300, 0.0, 300.0],
                         [0.0, 300, 300],
                         [0, 0, 1]], dtype=np.float32)

            rgb1 = read_rgb(rgb_file1_mpc)
            # Note that the depth values below are normalized to between 0 and 1. 
            depth1 = read_depth(depth_file1_mpc)
            transformation = np.eye(4)
            transformation[0,3] = translation_arr[x]
            rendered_image = get_rendered_image(depth1, rgb1, transformation, K)
            rendered_image = (rendered_image *255).astype(np.uint8)        
            im = pil.fromarray(rendered_image)
            if x % 2 == 0:
                save_folder = "right"
                if x != 0:
                    save_folder = "right{}".format(int(x/2)+1)
            elif x % 2 != 0:
                save_folder = "left"
                if x != 1:
                    save_folder = "left{}".format(int(int(x-1)/2)+1)

            save_path = os.path.join(FOLDER_out, position, save_folder, id_list_frames[fr][:-4] + ".png".format(x))
            if not os.path.exists(os.path.join(FOLDER_out, position)):
                os.mkdir(os.path.join(FOLDER_out, position))
            if not os.path.exists(os.path.join(FOLDER_out, position, save_folder)):
                os.mkdir(os.path.join(FOLDER_out, position, save_folder))

            #Resize to original size                              
            im = im.resize((1200,600))
                              
            #Center crop
            width, height = im.size   # Get dimensions
            new_width = 1000
            new_height = 600

            rgb = np.array(im)[:,:,:3]
            left = (width - new_width)/2
            top = (height - new_height)/2
            right = (width + new_width)/2
            bottom = (height + new_height)/2
            im = im.crop((left, top, right, bottom))
            im = im.resize((128, 128))
            rgb = np.array(im)[:,:,:3]
            
            im = pil.fromarray(rgb)
            im.save(save_path)
            print(save_path)
 
         
